# AESTHEMOS
For use with data collected from 

## TO-DO:

### add ids to paintings
### make function for computation and loop through all output files

AESTHEMOS (based on this):
This is the meat of this project. This is a measure that evaluates the subject's aesthetic experiences of each of the paintings they view. There should be about 66-67 paintings they view, and for 8 different categories of emotion, the subject gives a score between 1-7 based on intensity of that emotion.
There are 7 different form files, one for each different gallery (because it was too big to put in one form ): They are in questionnaires>Google Forms>AESTHEMOS
We want to divide all the paintings into one of 5 categories based on each individual's ratings: High Arousal Positive Valence, High Arousal Negative Valence, Low Arousal Positive Valence, Low Arousal Negative Valence, and Indifferent. This is calculated by:
Sort the paintings by the "Indifference" score. The top 10 with the highest Indifference scores will be part of the 'indiff' group.
Take out the 'indiff' group. Using the remaining paintings, create an "Arousal" measure which is the sum of ['a. Moved']+df['b. Fascinated']+df['c. Funny']+df['d. Surprised']
Sort paintings by Arousal; split in half. The ones with higher arousal scores go in the high arousal group, the ones with lower arousal scores go in the low arousal group.
Within each of those arousal groups, sort by "Unsettling". Split in half. If the Unsettled score is low, it's in the positive group, if it's high, then it's in the negative group. 
At this point you should have the 5 groups: hiPos, hiNeg, loPos, loNeg, and indiff. Create a csv that has a row for each painting and these columns: Painting Title, Artist, Gallery number, Unique ID, and the name of the Group it's in, as well as a column for each of the original measures on their ratings. I've attached a file called 'pilot02-groups-2-titles.csv' for reference; ultimately we want a file for each participant that looks like that but also includes all of the original ratings. The name of the file should be 'pilot__-groups.csv'
I have a Python script that I've used in the past that I can share with you if you like, but it was written for a different style of input (not a Google Form export) and also it's a mess haha. 

In [2]:
import pandas as pd
from numpy import array
import os

In [3]:
# Fixes numerical values in gallery files AND makes unique subject list
unique_subjects = set()
for gallery in os.listdir('data/aesthemos'):
    if gallery.endswith(".csv"):
        g = pd.read_csv("data/aesthemos/" + gallery).replace({'1 - Not at all':1, '7 - Strongly':7})
        g_subs = g['Subject #:'].tolist()
        z = [x.replace(' ', '').lower() for x in g_subs]
        unique_subjects = unique_subjects.union(set(z))
        g.to_csv("data/aesthemos/" + gallery, index=False)
list(unique_subjects)

['guy3', 'pilot02', 'pilot01']

In [4]:
# Setup key for painting IDs
IDsTable = pd.read_csv("paintingIDsKey.csv", encoding='latin1')
IDsTable

,Title,Artist,Unique ID
0,Composition with Boats,Stuart Davis,1005
1,Untitled,Janet Sobel,1014
2,Still-life Synchromy with Nude in Yellow,Morgan Russell,1013
3,Untitled,George Copeland Ault,1001
4,Morning News,Francis Luis Mora,1010
5,Mukie,Robert Henri,1009
6,Formation I,Arthur Garfield Dove,1006
7,Pool in the Mountains,Milton Avery,1002
8,"Late Afternoon, Giverny",Guy Orlando Rose,1012
9,The Edge of the Forest,Theodore Robinson,1011


In [5]:
for gallery in os.listdir('data/aesthemos'):
    if gallery.endswith(".csv"):
        print(gallery.split()[1].replace(".csv", ""))

019
003
002
016
200
001
100


In [6]:
raw_data = pd.read_csv("data/aesthemos/Gallery 003.csv")
raw_data

,Timestamp,Subject #:,Woman Seated by Hans Hofmann [Moved],Woman Seated by Hans Hofmann [Fascinated],Woman Seated by Hans Hofmann [Funny],Woman Seated by Hans Hofmann [Surprised],Woman Seated by Hans Hofmann [Indifferent],Woman Seated by Hans Hofmann [Calm],Woman Seated by Hans Hofmann [Unsettled],Woman Seated by Hans Hofmann [Beautiful],...,Still-life with Peaches by Raphaelle Peale [Unsettled],Still-life with Peaches by Raphaelle Peale [Beautiful],Still-life by José Agustin Arrieta [Moved],Still-life by José Agustin Arrieta [Fascinated],Still-life by José Agustin Arrieta [Funny],Still-life by José Agustin Arrieta [Surprised],Still-life by José Agustin Arrieta [Indifferent],Still-life by José Agustin Arrieta [Calm],Still-life by José Agustin Arrieta [Unsettled],Still-life by José Agustin Arrieta [Beautiful]
0,2020/04/07 2:00:48 PM MDT,pilot01,4,5,5,5,3,3,1,4,...,1,5,3,3,2,2,5,4,7,3
1,2020/04/08 1:27:20 PM MDT,pilot02,1,1,4,2,3,4,2,2,...,1,5,3,4,1,4,2,5,1,4


In [7]:
def clean_single_gallery(filesource, sub):
    file = pd.read_csv(filesource)
    sub_ser = pd.Series([x.replace(' ', '').lower() for x in file['Subject #:']])
    
    table = file.loc[sub_ser == sub].T.reset_index().drop([0,1]).rename(columns={"index": "Title"})
    if len(table.columns) > 1:
        table = (
            table
            .assign(category=table.Title.apply(lambda x: x.split(" ")[-1]))
            .assign(Name=table.Title.apply(lambda x: " ".join(x.split(" ")[:-1])))
            .rename(columns={ table.columns[-1]: "score"})
            .drop('Title', axis=1) 
         )
        table = table.assign(score=table.score.astype(int))
        t = table.pivot_table(index='Name', columns='category', aggfunc=(lambda x: x))
        t.columns = t.columns.levels[1].rename(None)
        t = t.rename(columns=lambda x: x[1:-1]).reset_index()
        art_split = t.Name.str.split(" by ", expand=True)
        t['Title'] = art_split[0]
        t['Artist'] = art_split[1]
        t = t.drop('Name', axis=1)
        t = pd.merge(t, IDsTable, how='left', on=['Title', 'Artist']) #SOME PAINTINGS DISSAPEAR
        return t
    else:
        print("ERROR: " + sub + " not in " + filesource)
        return;

In [8]:
# CONVERT ALL GALLERY FILES INTO SEPERATE SUBJECT CSVS
for subject in unique_subjects:
    current_sub = pd.DataFrame(
        columns = ["Title", "Artist" "Beautiful", "Calm", "Fascinated", "Funny", "Indifferent", "Moved", "Surprised", "Unsettled"]
    )
    for gallery in os.listdir('data/aesthemos'):
        if gallery.endswith(".csv"):
            fix_curr = clean_single_gallery("data/aesthemos/" + gallery, subject)
            if fix_curr is not None:
                fix_curr['Gallery'] = gallery.split()[1].replace(".csv", "")
                current_sub = current_sub.append(fix_curr, ignore_index = True, sort=True)
    current_sub.to_csv("output/"+ subject + "-groups.csv", index=False)
        

ERROR: guy3 not in data/aesthemos/Gallery 019.csv
ERROR: guy3 not in data/aesthemos/Gallery 003.csv
ERROR: guy3 not in data/aesthemos/Gallery 002.csv
ERROR: guy3 not in data/aesthemos/Gallery 016.csv
ERROR: guy3 not in data/aesthemos/Gallery 200.csv
ERROR: guy3 not in data/aesthemos/Gallery 100.csv


In [9]:
fix_curr

,Beautiful,Calm,Fascinated,Funny,Indifferent,Moved,Surprised,Unsettled,Title,Artist,Unique ID,Gallery
0,4,6,4,2,4,5,4,2,Afternoon Stroll,Chase,10003.0,100
1,4,5,4,4,4,5,4,1,Eye of Saint John,Hurd,NaN,100
2,3,4,4,5,4,4,4,1,Game of Croquet,Fries,10008.0,100
3,4,5,4,2,4,4,2,1,Rock ledge in Autumn,Braun,10007.0,100
4,4,5,5,2,4,5,4,1,San Felipe Valley,Reiffel,10004.0,100
5,5,5,5,2,3,5,4,1,Sierra Lake,Payne,10009.0,100
6,4,5,4,2,4,5,2,1,Southern California Landscape,Braun,10002.0,100
7,4,5,4,2,5,5,3,1,The White Flower,O'Keefe,10001.0,100
8,6,1,6,3,1,6,6,1,Torrey Pines,Fechin,10010.0,100


# Group Computation/Column Cleanup

In [10]:
# for loop processing each file to add computation stuff needed for catagories

In [11]:
fs = ("output/"+ "pilot01" + "-groups.csv")
file = pd.read_csv(fs)
df = file.sort_values("Indifferent") #lots of indifferent scores are the same...how do you want to further seperate
indiff = df.tail(10)
others = df.iloc[:-10]
print(len(df), len(indiff), len(others))

66 10 56


In [12]:
indiff["Group"] = "indiff"
indiff

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Artist,ArtistBeautiful,Beautiful,Calm,Fascinated,Funny,Gallery,Indifferent,Moved,Surprised,Title,Unique ID,Unsettled,Group
50,Francis Luis Mora,NaN,5.0,5,5,3,1,5,5,5,Morning News,1010.0,1,indiff
49,Guy Orlando Rose,NaN,4.0,5,4,2,1,5,4,2,"Late Afternoon, Giverny",NaN,1,indiff
6,José Agustin Arrieta,NaN,3.0,4,3,2,3,5,3,2,Still-life,NaN,7,indiff
9,George Wesley,NaN,4.0,6,5,4,3,5,5,4,Three Pigs and a Mountain,NaN,1,indiff
8,Hughie Lee-Smith,NaN,3.0,6,4,2,3,5,5,2,Sunday Afternoon,3005.0,2,indiff
54,Theodore Robinson,NaN,4.0,3,4,3,1,5,5,3,The Edge of the Forest,1011.0,1,indiff
19,Sarah Miriam Peale,NaN,4.0,5,4,2,2,5,5,2,Portrait of Mrs. William Crane,2011.0,1,indiff
47,Arthur Garfield Dove,NaN,4.0,7,2,3,1,6,2,2,Formation I,1006.0,1,indiff
18,George Caleb Bingham,NaN,4.0,6,2,2,2,6,3,2,"Portrait of Colonel James Hervey Birch, Jr.",2002.0,1,indiff
17,Eduard Jean Steichen,NaN,4.0,6,4,2,2,6,4,2,Portrait in Grey and Black,2012.0,1,indiff


In [23]:
indiff[indiff["Calm"] > 5]

,Artist,ArtistBeautiful,Beautiful,Calm,Fascinated,Funny,Gallery,Indifferent,Moved,Surprised,Title,Unique ID,Unsettled,Group
9,George Wesley,NaN,4.0,6,5,4,3,5,5,4,Three Pigs and a Mountain,NaN,1,indiff
8,Hughie Lee-Smith,NaN,3.0,6,4,2,3,5,5,2,Sunday Afternoon,3005.0,2,indiff
47,Arthur Garfield Dove,NaN,4.0,7,2,3,1,6,2,2,Formation I,1006.0,1,indiff
18,George Caleb Bingham,NaN,4.0,6,2,2,2,6,3,2,"Portrait of Colonel James Hervey Birch, Jr.",2002.0,1,indiff
17,Eduard Jean Steichen,NaN,4.0,6,4,2,2,6,4,2,Portrait in Grey and Black,2012.0,1,indiff


In [29]:
# create arousal (Moved + Fascinated + Funny + Surprised)
arousal = others['Moved'] + others['Fascinated'] + others['Funny'] + others['Surprised']
others["Arousal"] = arousal


# add high/low column to others
others = others.sort_values("Arousal")
others

,Artist,ArtistBeautiful,Beautiful,Calm,Fascinated,Funny,Gallery,Indifferent,Moved,Surprised,Title,Unique ID,Unsettled,Arousal
48,Daniel Garber,NaN,1.0,1,1,1,1,1,1,1,Junior Camp,1007.0,1,4
20,Emil J. Carlsen,NaN,1.0,1,1,1,2,1,1,1,Still-life,2003.0,7,4
23,Vincenzo Catena (Vincenzo di Biagio),NaN,4.0,5,3,2,16,3,4,2,Holy Family with Saint Anne,NaN,1,11
27,Fransesco Vecellio,NaN,4.0,6,3,2,16,4,4,2,Madonna and Child with Infant John the Baptist,1624.0,1,11
60,Braun,NaN,4.0,5,4,2,100,4,4,2,Rock ledge in Autumn,10007.0,1,12
16,Henry Inman,NaN,4.0,6,4,2,2,5,4,2,Mr. Paff,2007.0,1,12
36,Hieronymus Bosch,NaN,3.0,3,3,3,16,3,3,3,The Arrest of Christ,1606.0,6,12
45,Bellotto,NaN,4.0,6,4,2,200,4,4,3,"The Molo from the Basin of San Marco, Venice",NaN,1,13
63,Braun,NaN,4.0,5,4,2,100,4,5,2,Southern California Landscape,10002.0,1,13
11,Emil J. Carlsen,NaN,4.0,6,4,2,2,3,5,2,"Blue, White and Gold",2019.0,1,13


In [35]:
### middle value split
"""
low = others.head(int(len(others)/2))
low["hilo"] = "lo"
high = others.tail(int(len(others)/2))
high["hilo"] = "hi"
"""

### Mean value split
"""
aro_mean = others["Arousal"].mean() + 0.00001 # making non int should eliminate = mean
low = others[others["Arousal"] < aro_mean]
low["hilo"] = "lo"
high = others[others["Arousal"] > aro_mean]
high["hilo"] = "hi
""""


full_aro = pd.concat([low, high]).sort_values("Unsettled")
full_aro

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Artist,ArtistBeautiful,Beautiful,Calm,Fascinated,Funny,Gallery,Indifferent,Moved,Surprised,Title,Unique ID,Unsettled,Arousal,hilo
48,Daniel Garber,NaN,1.0,1,1,1,1,1,1,1,Junior Camp,1007.0,1,4,lo
5,Thomas Hart Benton,NaN,4.0,3,5,3,3,3,5,4,After Many Days,3003.0,1,17,hi
29,Pseudo Pier Francesco Fiorentino,NaN,4.0,5,4,4,16,4,5,4,Madonna of the Roses,1619.0,1,17,hi
58,Hurd,NaN,4.0,5,4,4,100,4,5,4,Eye of Saint John,NaN,1,17,hi
59,Fries,NaN,3.0,4,4,5,100,4,4,4,Game of Croquet,10008.0,1,17,hi
7,Raphaelle Peale,NaN,5.0,4,5,2,3,3,5,5,Still-life with Peaches,3007.0,1,17,hi
43,Sorolla y Bastida,NaN,5.0,3,5,2,200,3,5,5,Maria at La Granja,20002.0,1,17,hi
25,Carlo Crivelli,NaN,5.0,5,5,4,16,3,5,4,Madonna and Child,1609.0,1,18,hi
12,Thomas Easkins,NaN,4.0,5,4,5,2,3,5,4,Elizabeth Crowell with a Dog,2006.0,1,18,hi
0,Maximilien Luce,NaN,4.0,4,5,3,19,3,5,5,Notre Dame,1906.0,1,18,hi


In [36]:
# make pos/neg based off of Unsettled from full_aro

### middle value split
"""
neg = full_aro.head(int(len(full_aro)/2))
neg["PosNeg"] = "Neg"
pos = full_aro.tail(int(len(full_aro)/2))
pos["PosNeg"] = "Pos"
"""

### Mean value split
"""
unset_mean = full_aro["Unsettled"].mean() + 0.00001 # making non int should eliminate = mean
neg = full_aro[full_aro["Unsettled"] < unset_mean]
neg["PosNeg"] = "Neg"
pos = full_aro[full_aro["Unsettled"] > unset_mean]
pos["PosNeg"] = "Pos"
"""
pos

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Artist,ArtistBeautiful,Beautiful,Calm,Fascinated,Funny,Gallery,Indifferent,Moved,Surprised,Title,Unique ID,Unsettled,Arousal,hilo,PosNeg
38,Martin Bernat,NaN,4.0,4,3,4,16,4,4,3,The Crucifixion,NaN,2,14,lo,Pos
57,Chase,NaN,4.0,6,4,2,100,4,5,4,Afternoon Stroll,10003.0,2,15,lo,Pos
34,The Astorga Master,NaN,3.0,5,4,5,16,4,4,4,Saint Peter,1601.0,2,17,hi,Pos
36,Hieronymus Bosch,NaN,3.0,3,3,3,16,3,3,3,The Arrest of Christ,1606.0,6,12,lo,Pos
20,Emil J. Carlsen,NaN,1.0,1,1,1,2,1,1,1,Still-life,2003.0,7,4,lo,Pos


In [189]:
#merge hilow/posneg and make groups col
final_df = pd.concat([pos, neg])
final_df["Group"] = final_df["hilo"] + final_df["PosNeg"]

final_df = final_df.drop(columns=["hilo", "PosNeg"])
final_df

,Artist,ArtistBeautiful,Beautiful,Calm,Fascinated,Funny,Gallery,Indifferent,Moved,Surprised,Title,Unique ID,Unsettled,Arousal,Group
61,Reiffel,NaN,4.0,5,5,2,100,4,5,4,San Felipe Valley,10004.0,1,16,loPos
23,Vincenzo Catena (Vincenzo di Biagio),NaN,4.0,5,3,2,16,3,4,2,Holy Family with Saint Anne,NaN,1,11,loPos
27,Fransesco Vecellio,NaN,4.0,6,3,2,16,4,4,2,Madonna and Child with Infant John the Baptist,1624.0,1,11,loPos
60,Braun,NaN,4.0,5,4,2,100,4,4,2,Rock ledge in Autumn,10007.0,1,12,loPos
16,Henry Inman,NaN,4.0,6,4,2,2,5,4,2,Mr. Paff,2007.0,1,12,loPos
45,Bellotto,NaN,4.0,6,4,2,200,4,4,3,"The Molo from the Basin of San Marco, Venice",NaN,1,13,loPos
63,Braun,NaN,4.0,5,4,2,100,4,5,2,Southern California Landscape,10002.0,1,13,loPos
11,Emil J. Carlsen,NaN,4.0,6,4,2,2,3,5,2,"Blue, White and Gold",2019.0,1,13,loPos
2,Georges Braque,NaN,3.0,6,3,4,19,4,4,2,The Corn-poppies,1903.0,1,13,loPos
35,de Ribera,NaN,4.0,4,3,4,16,4,4,3,Susanna and the Elders,1628.0,1,14,loPos


In [190]:
# Add indiff and order columns
final_df = final_df.append(indiff, ignore_index = True)
cols = ["Title", "Artist", "Gallery", "Unique ID", "Moved", "Fascinated", "Funny", "Surprised", 
        "Indifferent", "Calm", "Unsettled", "Beautiful", "Arousal", "Group"]
# SEE IF AROUSAL IS NEEDED, ADD "Unique ID" AFTER GALLERY
final_df = final_df[cols]

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [191]:
final_df

,Title,Artist,Gallery,Unique ID,Moved,Fascinated,Funny,Surprised,Indifferent,Calm,Unsettled,Beautiful,Arousal,Group
0,San Felipe Valley,Reiffel,100,10004.0,5,5,2,4,4,5,1,4.0,16.0,loPos
1,Holy Family with Saint Anne,Vincenzo Catena (Vincenzo di Biagio),16,NaN,4,3,2,2,3,5,1,4.0,11.0,loPos
2,Madonna and Child with Infant John the Baptist,Fransesco Vecellio,16,1624.0,4,3,2,2,4,6,1,4.0,11.0,loPos
3,Rock ledge in Autumn,Braun,100,10007.0,4,4,2,2,4,5,1,4.0,12.0,loPos
4,Mr. Paff,Henry Inman,2,2007.0,4,4,2,2,5,6,1,4.0,12.0,loPos
5,"The Molo from the Basin of San Marco, Venice",Bellotto,200,NaN,4,4,2,3,4,6,1,4.0,13.0,loPos
6,Southern California Landscape,Braun,100,10002.0,5,4,2,2,4,5,1,4.0,13.0,loPos
7,"Blue, White and Gold",Emil J. Carlsen,2,2019.0,5,4,2,2,3,6,1,4.0,13.0,loPos
8,The Corn-poppies,Georges Braque,19,1903.0,4,3,4,2,4,6,1,3.0,13.0,loPos
9,Susanna and the Elders,de Ribera,16,1628.0,4,3,4,3,4,4,1,4.0,14.0,loPos


In [192]:
# Setup sort for rows so that it knows order Group should go in
final_df["Group"] = pd.Categorical(final_df['Group'], ["hiPos", "hiNeg", "loPos", "loNeg", "indiff"])
final_df = final_df.sort_values(['Group','Gallery'])
final_df

,Title,Artist,Gallery,Unique ID,Moved,Fascinated,Funny,Surprised,Indifferent,Calm,Unsettled,Beautiful,Arousal,Group
12,Still-life Synchromy with Nude in Yellow,Morgan Russell,1,1013.0,6,7,5,6,1,3,1,6.0,24.0,hiPos
24,Saint Peter,The Astorga Master,16,1601.0,4,4,5,4,4,5,2,3.0,17.0,hiPos
38,Composition with Boats,Stuart Davis,1,1005.0,5,5,5,3,4,4,1,5.0,18.0,hiNeg
43,Untitled,George Copeland Ault,1,1001.0,5,5,5,4,3,5,1,5.0,19.0,hiNeg
49,Mukie,Robert Henri,1,1009.0,5,5,5,5,2,5,1,6.0,20.0,hiNeg
50,Pool in the Mountains,Milton Avery,1,1002.0,5,5,5,5,2,4,1,4.0,20.0,hiNeg
45,"Farm Landscape, Cattle in Pasture--Sunset Nant...",George Inness,2,2008.0,5,5,5,4,3,6,1,4.0,19.0,hiNeg
53,Elizabeth Crowell with a Dog,Thomas Easkins,2,2006.0,5,4,5,4,3,5,1,4.0,18.0,hiNeg
54,"Kilauea Caldera, Sandwich Islands",Jules Tavernier,2,2013.0,6,6,5,5,2,3,1,6.0,22.0,hiNeg
29,After Many Days,Thomas Hart Benton,3,3003.0,5,5,3,4,3,3,1,4.0,17.0,hiNeg


In [193]:
fs2 = ("output/"+ "pilot01" + "-groupsFINALTEST.csv")
final_df.to_csv(fs2, index=False)

In [63]:
def evaluate_subject_paintings(filesource):
    file = pd.read_csv(filesource)
    
    
    current_sub.to_csv(filesource, index=False)

In [ ]:
for subject in unique_subjects:
    evaluate_subject_paintings("output/"+ subject + "-groups.csv")